In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

w:\anaconda\projectE2E\Regression_ML_EndtoEnd\.venv\Scripts\python.exe
3.0.4
w:\anaconda\projectE2E\Regression_ML_EndtoEnd\.venv\Lib\site-packages\xgboost\__init__.py


In [3]:
# /Users/riadanas/Desktop/housing regression MLE/.venv/bin/python
# 3.0.4
# /Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/xgboost/__init__.py

In [5]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

In [6]:
# ==============================================
# 2. Load processed datasets
# ==============================================
# train_df = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered_train.csv")
# eval_df  = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered_eval.csv")

train_df = pd.read_csv("W:/anaconda/projectE2E/Regression_ML_EndtoEnd/data/processed/feature_engineered_train.csv")
eval_df = pd.read_csv("W:/anaconda/projectE2E/Regression_ML_EndtoEnd/data/processed/feature_engineered_eval.csv")
# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (585199, 39)
Eval shape: (148448, 39)


In [7]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [11]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder

# mlflow.set_tracking_uri("/Users/riadanas/Desktop/housing regression MLE/mlruns")
mlflow.set_tracking_uri("/anaconda/projectE2E/Regression_ML_EndtoEnd/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=25)

print("Best params:", study.best_trial.params)

[I 2025-12-06 20:11:19,947] A new study created in memory with name: no-name-2c428286-626a-40c4-89db-9fd4febe1c09
[I 2025-12-06 20:11:29,302] Trial 0 finished with value: 75460.56541763061 and parameters: {'n_estimators': 598, 'max_depth': 3, 'learning_rate': 0.08817584369303157, 'subsample': 0.7898587142961873, 'colsample_bytree': 0.6549509766517527, 'min_child_weight': 3, 'gamma': 2.5982314030536298, 'reg_alpha': 1.2764982000741476e-08, 'reg_lambda': 0.013032630735377295}. Best is trial 0 with value: 75460.56541763061.
[I 2025-12-06 20:11:50,482] Trial 1 finished with value: 72112.72317985534 and parameters: {'n_estimators': 877, 'max_depth': 7, 'learning_rate': 0.02239099604836842, 'subsample': 0.6684952008573235, 'colsample_bytree': 0.8370108236475617, 'min_child_weight': 10, 'gamma': 0.963427932354336, 'reg_alpha': 0.5146234907285727, 'reg_lambda': 0.12243678308601519}. Best is trial 1 with value: 72112.72317985534.
[I 2025-12-06 20:11:59,726] Trial 2 finished with value: 74584.69

Best params: {'n_estimators': 732, 'max_depth': 7, 'learning_rate': 0.026986955979827296, 'subsample': 0.7667719768904239, 'colsample_bytree': 0.5831358464201443, 'min_child_weight': 9, 'gamma': 1.8767683770881196, 'reg_alpha': 7.3108055341415e-05, 'reg_lambda': 1.1630924002402886e-08}


In [12]:
best_params = study.best_trial.params
print(best_params)

{'n_estimators': 732, 'max_depth': 7, 'learning_rate': 0.026986955979827296, 'subsample': 0.7667719768904239, 'colsample_bytree': 0.5831358464201443, 'min_child_weight': 9, 'gamma': 1.8767683770881196, 'reg_alpha': 7.3108055341415e-05, 'reg_lambda': 1.1630924002402886e-08}


In [ ]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model           
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 31761.271821385097
RMSE: 71129.96844700849
R²: 0.9609010348146156


AttributeError: module 'mlflow' has no attribute 'MlflowClient'

In [ ]:
Final tuned model performance:
MAE: 31472.002185832847
RMSE: 70459.0823185452
R²: 0.9616351065691122